In [1]:

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from collections import OrderedDict
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.nn.functional import normalize
# from tensorflow.keras.datasets import mnist
# from imutils import build_montages
import numpy as np
# import cv2

import pandas as pd
import re
import os
import random
from osgeo import gdal
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings("ignore")
%matplotlib inline

torch.cuda.empty_cache()


c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# GPU related info
cuda = 1
device = torch.device("cuda" if torch.cuda.is_available() and cuda == 1 else "cpu") # default gpu
print("Device:", device)

Device: cpu


In [3]:
path="data/"
# path_m="../../../../Dataset/fusion_modis_landsat"

In [4]:
# landsat_folder=path+"/landsat_5_1_2014"
landsat_folder=path+"landsat"
# modis_folder=path+"/modis_5_1_2014"
modis_folder=path+"modis"
print(landsat_folder)
print(modis_folder)

data/landsat
data/modis


In [5]:
dir_list_ls = os.listdir(landsat_folder)
dir_list_md = os.listdir(modis_folder)
# type(dir_list)
len(dir_list_ls), len(dir_list_md)

(23, 46)

In [6]:
def unique(list1):
 
    # initialize a null list
    unique_list = []
 
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [7]:
def process_state(dir_list,sat):
    states = []
    counties = []
    
    for i in range(len(dir_list)):
        file = dir_list[i]
        locations=file[:-4].split("_")
#         print('location',locations) 
        if sat == 'ls':
            state = int(locations[1])
        if sat == 'md':
            state = int(locations[0])
        states.append(state)
        stat = unique(states)
#         print(stat)
    return stat
    
    


In [8]:
def process_county(dir_list, state, sat):
    states = []
    counties = []
    
    for i in range(len(dir_list)):
        file = dir_list[i]
        locations=file[:-4].split("_")
#         print('location',locations) 
        if sat == 'ls':
            state_fl = int(locations[1])
        if sat == 'md':
            state_fl = int(locations[0])

        if (state_fl != state):
            continue
        elif (state_fl == state and sat == 'ls'):
            county_fl = int(locations[2])   
        elif (state_fl == state and sat == 'md'):
            county_fl = int(locations[1])          
        counties.append(county_fl)
        county = unique(counties)            
            
            
            
#         if (state_fl == state and sat == 'ls'):
#             county_fl = int(locations[2])
#         if (state_fl == state and sat == 'md'):
#             county_fl = int(locations[1])            
#         counties.append(county_fl)
#         county = unique(counties)
    return county
    
    


In [9]:
def common(a,b): 
    c = [value for value in a if value in b] 
    return c

In [10]:
# states=[5]
# counties=[1,3,7,17,19,21,27,29,31,33,35,37,41,43,45,47,55,59,63,67,69,71,73,75,77,79,81,83,85,91,93,95,105,107,111,115,117,119,121,123,131,145,149]

test_years = [2014]
tst_year = test_years[0]
start_year = 2014
train_years = list (range(start_year,tst_year+1))
val_years = [tst_year, tst_year]

# print(val_years)

def getModisName(state, county, year, step):
    return str(state)+"_"+str(county)+"_"+str(year)+"_"+str(step)+".tif"
def getLandsatName(state, county, year, step):
    return "landsat_"+str(state).zfill(2)+"_"+str(county).zfill(3)+"_"+str(year)+"_"+str(step)+".tif"

modis_train = []
modis_val = []
modis_test = []

ls_train = []
ls_val = []
ls_test = []

states_ls = process_state(dir_list_ls,'ls')
states_md = process_state(dir_list_md,'md')

states = common(states_ls,states_md)


# print('state', states)
for state in states:
    counties_ls = process_county(dir_list_ls,state,'ls')
    counties_md = process_county(dir_list_md,state,'md')
    counties = common(counties_ls,counties_md)
    for county in counties:
        for year in range(start_year,(tst_year+1)):
#             modis_filename_test=getModisName(state, county, year, step)
#             landsat_filename_test=getLandsatName(state, county, year, step)
            for step in range(1,47,2):
                m_file=getModisName(state, county, year, step)
                if year in train_years:
                    modis_train.append(m_file)
                if year in val_years:
                    modis_val.append(m_file)
                if year in test_years:
                    modis_test.append(m_file)       
            for step in range(1,24):
                l_file=getLandsatName(state, county, year, step)
                if year in train_years:
                    ls_train.append(l_file)
                if year in val_years:
                    ls_val.append(l_file)
                if year == tst_year:
                    ls_test.append(l_file)             

print(len(modis_train),len(modis_val),len(modis_test))
print(len(ls_train),len(ls_val),len(ls_test))
print(modis_train)
print(modis_val)
print(modis_test)
print(ls_train)

23 23 23
23 23 23
['5_1_2014_1.tif', '5_1_2014_3.tif', '5_1_2014_5.tif', '5_1_2014_7.tif', '5_1_2014_9.tif', '5_1_2014_11.tif', '5_1_2014_13.tif', '5_1_2014_15.tif', '5_1_2014_17.tif', '5_1_2014_19.tif', '5_1_2014_21.tif', '5_1_2014_23.tif', '5_1_2014_25.tif', '5_1_2014_27.tif', '5_1_2014_29.tif', '5_1_2014_31.tif', '5_1_2014_33.tif', '5_1_2014_35.tif', '5_1_2014_37.tif', '5_1_2014_39.tif', '5_1_2014_41.tif', '5_1_2014_43.tif', '5_1_2014_45.tif']
['5_1_2014_1.tif', '5_1_2014_3.tif', '5_1_2014_5.tif', '5_1_2014_7.tif', '5_1_2014_9.tif', '5_1_2014_11.tif', '5_1_2014_13.tif', '5_1_2014_15.tif', '5_1_2014_17.tif', '5_1_2014_19.tif', '5_1_2014_21.tif', '5_1_2014_23.tif', '5_1_2014_25.tif', '5_1_2014_27.tif', '5_1_2014_29.tif', '5_1_2014_31.tif', '5_1_2014_33.tif', '5_1_2014_35.tif', '5_1_2014_37.tif', '5_1_2014_39.tif', '5_1_2014_41.tif', '5_1_2014_43.tif', '5_1_2014_45.tif']
['5_1_2014_1.tif', '5_1_2014_3.tif', '5_1_2014_5.tif', '5_1_2014_7.tif', '5_1_2014_9.tif', '5_1_2014_11.tif', '5_1_2

In [11]:
rc_m = torchvision.transforms.Resize((256, 256),interpolation=T.InterpolationMode.NEAREST) ##original 150,250
rc_l = torchvision.transforms.Resize((256, 256),interpolation=T.InterpolationMode.NEAREST) ##original (455, 575)

# rc_m = torchvision.transforms.Pad(91, 115) ##original 150,250
# rc_l = torchvision.transforms.Pad(2275, 2875) ##original 150,250

In [12]:
def masking(x,sat):
#     x = torch.tensor(x)
#     print(x.shape)
#     xreshape = x.reshape(x.shape[0],x.shape[1],x.shape[2])
#     print(xreshape.shape)
#     print('nan',img, torch.isnan(xreshape).sum())
#     print('inf',img, torch.isinf(xreshape).sum())
#     print('minmax', img, torch.min(xreshape),torch.max(xreshape) ) 
    x = x.to(device)
    xnan = torch.where(torch.isnan(x),0,x)
    if sat =='ls':
        xinf = torch.where(torch.isinf(xnan),1,xnan)
    else:
        xinf = torch.where(torch.isinf(xnan),5000,xnan)
#     print('nan',img, torch.isnan(xnan).sum())  
#     print('inf',img, torch.isinf(xnan).sum())      
#     print('minmax', img, torch.min(xnan),torch.max(xnan) )      
    cnnv= nn.AvgPool2d(kernel_size =3, stride=1, padding = 1, ceil_mode=False,count_include_pad=False)
    cnnv = cnnv.to(device)
    xinf = xinf.to(device)
    output = cnnv(xinf)
#     print(out.shape)
#     resin = torch.where(torch.isnan(x),output,x)
#     resin = torch.where(torch.isnan(x),xnan,x)
#     resinf = torch.where(torch.isinf(resin),xinf,resin) 
    resin = torch.where(torch.isnan(x),output,x)
    resinf = torch.where(torch.isinf(resin),output,resin) 
    res = torch.squeeze(resinf)
#     res = res.numpy()
#     print(res.shape)
    return res

In [13]:
class CustomImageDataset(Dataset):
    def __init__(self, modis_path, modis_list, ls_path, ls_list):
        self.modis_path = modis_path
        self.modis_list = modis_list
        self.ls_path    = ls_path
        self.ls_list    = ls_list


    def __len__(self):
        return len(self.modis_list)

    def __getitem__(self, idx):

        file_num = idx
    
        modis_file = self.modis_list[file_num]
        ls_file    = self.ls_list[file_num]        
        ls_file_prev = self.ls_list[file_num-1]
        modis_img_path = os.path.join(self.modis_path, modis_file)
        ls_img_path    = os.path.join(self.ls_path, ls_file) 
        ls_img_path_prev = os.path.join(self.ls_path, ls_file_prev)
        modis_img = gdal.Open(modis_img_path).ReadAsArray()
        ls_img = gdal.Open(ls_img_path).ReadAsArray() 
        ls_img_prev = gdal.Open(ls_img_path_prev).ReadAsArray()             
        modis_img = modis_img/255
        ls_img = ls_img/255
        ls_img_prev = ls_img_prev/255      
        
        modis_img = np.array(modis_img)
        ls_img    = np.array(ls_img)
        ls_img_prev = np.array(ls_img_prev)

        modis_img = torch.tensor(modis_img)
        ls_img    = torch.tensor(ls_img)
        ls_img_prev = torch.tensor(ls_img_prev)
        modis_img = masking(modis_img, 'md')
        ls_img = masking(ls_img, 'ls')
        ls_img_prev = masking(ls_img_prev, 'ls')
        modis_img = rc_m(modis_img)
        ls_img    = rc_l(ls_img)
        ls_img_prev = rc_l(ls_img_prev)
        source = torch.cat((modis_img,ls_img_prev),0)    
        return_value = [source,ls_img]     
        return return_value

In [14]:
dataset = CustomImageDataset(modis_folder,modis_train,landsat_folder,ls_train)


fnames = ['modis',  'ls'] 
# print(dataset[0])
item   = dataset[0]
print(item)
# print([(fnames[i],item[i].shape) for i in range(len(item))])


[tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3412, 0.3412, 0.8235],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.7882, 0.7882, 0.1294],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.5373, 0.5373, 0

In [15]:
train_data = CustomImageDataset(modis_folder,modis_train,landsat_folder,ls_train)
val_data   = CustomImageDataset(modis_folder,modis_val,landsat_folder,ls_val)
test_data  = CustomImageDataset(modis_folder,modis_test,landsat_folder,ls_test)
print(train_data[0])

[tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3412, 0.3412, 0.8235],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.7882, 0.7882, 0.1294],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.5373, 0.5373, 0

In [16]:
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [17]:
for batch_idx, (source, target)  in enumerate(train_dataloader):
    x= (batch_idx, source.shape,  target.shape)
    print (x)

(0, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(1, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(2, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(3, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(4, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(5, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(6, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(7, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(8, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(9, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(10, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(11, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(12, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(13, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(14, torch.Size([1, 10, 256, 256]), torch.Size([1, 5, 256, 256]))
(15, torch.Size([1, 

In [18]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [19]:
import numpy as np
import torch.nn as nn


class Conv2d(nn.Module):
    """Conv2d + BatchNorm + Dropout + ReLU
    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the convolution
        kernel_size (int or tuple): Size of the convolving kernel
        stride (int or tuple, optional): Stride of the convolution. Default: 1
        padding (int or tuple, optional): Zero-padding added to both sides of the input. Default: 0
        dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
        relu (bool, str): if True, uses ReLU - if 'learn', uses PReLU
        leak (float): if > 0 and relu == True, applies leaky ReLU instead
        bn (bool): if True, uses batch normalization
        dropout (float): dropout probability
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, bias=True, dilation=1, relu=False, leak=0.,
                 dropout=0., bn=False):
        super(Conv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=kernel_size,
                              stride=stride,
                              padding=padding,
                              dilation=dilation,
                              bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-5, momentum=0.1, affine=True) if bn else None
        self.dropout = nn.Dropout(p=dropout, inplace=True) if dropout > 0 else None
        if relu:
            if leak > 0:
                self.relu = nn.LeakyReLU(negative_slope=leak, inplace=True)
            elif relu == 'learn':
                self.relu = nn.PReLU()
            elif relu is True:
                self.relu = nn.ReLU(inplace=True)
            else:
                raise ValueError("Unknown argument specified for ReLU activation")
        else:
            self.relu = None

        # Weights initializer
        nn.init.xavier_normal_(self.conv.weight)
        if self.conv.bias is not None:
            nn.init.zeros_(self.conv.bias)

    def forward(self, x):
        x = self.conv(x)
        if self.bn:
            x = self.bn(x)
        if self.dropout:
            x = self.dropout(x)
        if self.relu:
            x = self.relu(x)
        return x

    def output_size(self, input_size):
        """Computes output size
        Args:
            input_size (tuple): (C_in, H_in, W_in)
        """
        _, H_in, W_in = input_size
        C_out = self.conv.out_channels
        kernel_size = self.conv.kernel_size[0]
        padding = self.conv.padding[0]
        stride = self.conv.stride[0]
        H_out = int(np.floor((H_in - kernel_size + 2 * padding) / stride + 1))
        W_out = int(np.floor((W_in - kernel_size + 2 * padding) / stride + 1))
        return (C_out, H_out, W_out)


class ConvTranspose2d(nn.Module):
    """Conv2d + BatchNorm + Dropout + ReLU
    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the convolution
        kernel_size (int or tuple): Size of the convolving kernel
        stride (int or tuple, optional): stride for the cross-correlation. Default: 1
        padding (int or tuple, optional): zero-padding will be added to both sides of each dimension in the inpu
        output_padding (int or tuple, optional): controls the additional size added to one side of the output shape. Default: 0
        dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
        relu (bool, str): if True, uses ReLU - if 'learn', uses PReLU
        leak (float): if > 0 and relu == True, applies leaky ReLU instead
        dropout (float): dropout probability
        bn (bool): if True, uses batch normalization
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=1, output_padding=0, bias=True, dilation=1, relu=False,
                 leak=0., dropout=0., bn=False):
        super(ConvTranspose2d, self).__init__()
        self.conv = nn.ConvTranspose2d(in_channels=in_channels,
                                       out_channels=out_channels,
                                       kernel_size=kernel_size,
                                       stride=stride,
                                       padding=padding,
                                       output_padding=output_padding,
                                       dilation=dilation,
                                       bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-5, momentum=0.1, affine=True) if bn else None
        self.dropout = nn.Dropout(p=dropout, inplace=True) if dropout > 0 else None
        if relu:
            if leak > 0:
                self.relu = nn.LeakyReLU(negative_slope=leak, inplace=True)
            elif relu == 'learn':
                self.relu = nn.PReLU()
            elif relu is True:
                self.relu = nn.ReLU(inplace=True)
            else:
                raise ValueError("Unknown argument specified for ReLU activation")
        else:
            self.relu = None

        # Weights initializer
        nn.init.xavier_normal_(self.conv.weight)
        if self.conv.bias is not None:
            nn.init.zeros_(self.conv.bias)

    def forward(self, x):
        x = self.conv(x)
        if self.bn:
            x = self.bn(x)
        if self.dropout:
            x = self.dropout(x)
        if self.relu:
            x = self.relu(x)
        return x

    def output_size(self, input_size):
        """Computes output size
        Args:
            input_size (tuple): (C, H_in, W_in)
        """
        raise NotImplementedError


class ResBlock(nn.Module):
    """2D-Convolutional residual unit

    # TODO : add ascii drawing of block

    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the convolution
        scaling (float): residual scaling factor
        stride (int or tuple, optional): Stride of the convolution. Default: 1
        bias (bool): if True, uses bias parameter in convolutions
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, scaling=0.1, bias=False, leak=0.):
        super().__init__()
        self.conv1 = Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride,
                            padding=padding, bias=bias, relu=True, leak=leak, bn=True)
        self.conv2 = Conv2d(out_channels, out_channels, kernel_size=3, stride=1,
                            padding=1, bias=bias, relu=False, bn=True)
        self.adjust_identity = None
        if stride > 1 or in_channels != out_channels:
            self.adjust_identity = Conv2d(in_channels, out_channels, kernel_size=1, stride=stride,
                                          bias=False, dilation=1, relu=False, bn=True)
        self.scaling = scaling
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        buffer = self.conv1(x)
        residual = self.conv2(buffer)

        if self.adjust_identity is not None:
            x = self.adjust_identity(x)

        residual = residual.mul(self.scaling)
        x = x.add(residual)
        x = self.relu(x)
        return x

In [20]:
def accuracy(predicted, groundtruth, thresh=0.5):
    """Accuracy on single label classification
    Args:
        predicted (torch.Tensor): batch of probability distributions on classes
        groundtruth (torch.Tensor): batch of probability distributions on classes
    """
    predicted = predicted > thresh
    correct = torch.sum(predicted.float() == groundtruth).float()
    score = correct / groundtruth.numel()
    return score.item()


def precision(predicted, groundtruth, thresh=0.5):
    """Precision on single label classification
    Args:
        predicted (torch.Tensor): batch of probability distributions on classes
        groundtruth (torch.Tensor): batch of probability distributions on classes
    """
    positives = predicted > thresh
    true_positives = torch.sum(positives[positives].float() == groundtruth[positives]).float()
    precision = true_positives.div(positives.sum())
    return precision.item()


def recall(predicted, groundtruth, thresh=0.5):
    """Recall on single label classification
    Args:
        predicted (torch.Tensor): batch of probability distributions on classes
        groundtruth (torch.Tensor): batch of probability distributions on classes
    """
    predicted = predicted > thresh
    positives = groundtruth == 1
    true_positives = torch.sum(predicted[positives].float() == groundtruth[positives]).float()
    recall = true_positives.div(positives.sum())
    return recall.item()

In [21]:
import numpy as np
from scipy import signal
from skimage import metrics


def psnr(ref, tgt):
    """Computes peak signal to noise ratio of a target image wrt to a
    reference image

    Args:
        ref (np.ndarray): Reference image as (height, width)
        tgt (np.ndarray): Target image as (height, width)

    Returns:
        type: float

    """
    return metrics.peak_signal_noise_ratio(image_true=ref,
                                           image_test=tgt)


def ssim(ref, tgt, window_size=None):
    """Computes mean structural similarity index between two images

    "Image quality assessment: from error visibility to structural similarity",
    Wang et. al 2004

    Args:
        ref (np.ndarray): Reference image as (height, width)
        tgt (np.ndarray): Target image as (height, width)
        window_size (int): side-length of the sliding window used in comparison
            must be odd value (default: None)

    Returns:
        type: float

    """
    return metrics.structural_similarity(im1=ref,
                                         im2=tgt,
                                         win_size=window_size,data_range=tgt.max()-tgt.min())


def sam(ref, tgt, reduce=None):
    """Computes normalized Spectrale Angle Mapper as introduced in

    "Discrimination among semi-arid landscape endmembers using the spectral
    angle mapper (SAM) algorithm", Boardman et al. 1993

    Args:
        ref (np.ndarray): Reference image as (height, width, channels)
        tgt (np.ndarray): Target image as (height, width, channels)
        reduce (str): output reduction method

    Returns:
        type: {np.ndarray, float}
    """
    # Compute pixelwise bands inner product
    kernel = np.einsum('...k,...k', ref, tgt)

    # Normalize inner products
    square_norm_ref = np.einsum('...k,...k', ref, ref).clip(min=np.finfo(np.float16).eps)
    square_norm_tgt = np.einsum('...k,...k', tgt, tgt).clip(min=np.finfo(np.float16).eps)
    normalized_kernel = kernel / np.sqrt(square_norm_ref * square_norm_tgt)

    # Convert to angles
    normalized_angles = np.arccos(normalized_kernel.clip(min=-1, max=1)) / np.pi
    if not reduce:
        output = normalized_angles
    elif reduce == 'mean':
        output = normalized_angles.mean()
    else:
        raise ValueError("Unknown reduce method")
    return output


def cw_ssim(ref, tgt, width=30, K=0.01):
    """Compute the complex wavelet SSIM (CW-SSIM) value from the reference
    image to the target image.

    "CW-SSIM based image classification", Gao et. al 2011

    Code based on https://github.com/jterrace/pyssim/blob/master/ssim/ssimlib.py

    Args:
      ref (np.ndarray): Reference image as (height, width)
      tgt (np.ndarray): Target image as (height, width)
      width (int): width for the wavelet convolution (default: 30)
      K (float): small algorithm parameter (default: 0.01)
    Returns:
      type: float
    """
    # Define a width for the wavelet convolution
    widths = np.arange(1, width + 1)

    # Convolution
    cwt1 = signal.cwt(ref.flatten(), signal.ricker, widths)
    cwt2 = signal.cwt(tgt.flatten(), signal.ricker, widths)

    # Compute the first term
    c1c2 = np.abs(cwt1) * np.abs(cwt2)
    c1_2 = np.square(np.abs(cwt1))
    c2_2 = np.square(np.abs(cwt2))
    numerator_ssim1 = 2 * c1c2.sum(axis=0) + K
    denominator_ssim1 = c1_2.sum(axis=0) + c2_2.sum(axis=0) + K

    # Compute the second term
    c1c2_conj = cwt1 * np.conjugate(cwt2)
    numerator_ssim2 = 2 * np.abs(c1c2_conj.sum(axis=0)) + K
    denominator_ssim2 = 2 * np.abs(c1c2_conj).sum(axis=0) + K

    # Construct the result
    ssim_map = (numerator_ssim1 / denominator_ssim1) * (numerator_ssim2 / denominator_ssim2)

    # Average the per pixel results
    index = np.mean(ssim_map)
    return index

In [22]:
def _compute_classification_metrics(output_real_sample, output_fake_sample):
        """Computes metrics on discriminator classification power : fooling rate
            of generator, precision and recall

        Args:
            output_real_sample (torch.Tensor): discriminator prediction on real samples
            output_fake_sample (torch.Tensor): discriminator prediction on fake samples

        Returns:
            type: tuple[float]
        """
        # Setup complete outputs and targets vectors
        target_real_sample = torch.ones_like(output_real_sample)
        target_fake_sample = torch.zeros_like(output_fake_sample)
        output = torch.cat([output_real_sample, output_fake_sample])
        target = torch.cat([target_real_sample, target_fake_sample])

        # Compute generator and discriminator metrics
        fooling_rate = accuracy(output_fake_sample, target_real_sample)
        precision1 = precision(output, target)
        recall1 = recall(output, target)
        return fooling_rate, precision1, recall1

In [23]:
import pytorch_lightning as pl
import torch
from torch.utils.data import random_split
import numpy as np
from collections import defaultdict
def _compute_iqa_metrics(estimated_target, target, reduction=None):
        """Computes full reference image quality assessment metrics : psnr, ssim
            and spectral angle mapper (see evaluation/metrics/iqa.py for details)

        Args:
            estimated_target (torch.Tensor): generated sample
            target (torch.Tensor): target sample

        Returns:
            type: tuple[float]
        """
        # Reshape as (batch_size * channels, height, width) to run single for loop
        batch_size, channels, height, width = target.shape
        estimated_target = estimated_target.clamp(min=0).detach().cpu().numpy()
        target = target.clamp(min=0).detach().cpu().numpy()
        estimated_bands = estimated_target.reshape(-1, height, width)
        target_bands = target.reshape(-1, height, width)

        # Compute IQA metrics by band
        iqa_metrics = defaultdict(list)
        for src, tgt in zip(estimated_bands, target_bands):
            data_range = np.max([src, tgt])
            src = src / data_range
            tgt = tgt / data_range
            iqa_metrics['psnr'] += [psnr(tgt, src)]
            iqa_metrics['ssim'] += [ssim(tgt, src)]

        # Average bandwise - reduce if needed
        psnr1 = np.asarray(iqa_metrics['psnr']).reshape(batch_size, channels).mean(axis=0)
        ssim1 = np.asarray(iqa_metrics['ssim']).reshape(batch_size, channels).mean(axis=0)
        if reduction == 'mean':
            psnr1 = psnr1.mean()
            ssim1 = ssim1.mean()

        # Compute average spectral angle mapper
        estimated_target = estimated_target.transpose(0, 2, 3, 1).astype(np.float32)
        target = target.transpose(0, 2, 3, 1).astype(np.float32)
        sam1 = sam(target, estimated_target).mean()
        return psnr1, ssim1, sam1

In [54]:
###   for bigger size of landsat ###########


class Generator(nn.Module):
    def __init__(self,landsat_inc):
        super(Generator, self).__init__()
        
        #Encoder
        self.conv1 = Conv2d(landsat_inc*2,64,4,2,1,bn=False,relu=False)
        print(self.conv1)
        self.conv2 = Conv2d(64,128,4,2,1,bn=True,relu="learn")
        self.conv3 = Conv2d(128,256,4,2,1,bn=True,relu="learn")
        self.conv4 = Conv2d(256,512,4,2,1,bn=True,relu="learn")
        self.conv5 = Conv2d(512,1024,4,2,1,bn=True,relu="learn")
        self.conv6 = Conv2d(1024,1024,4,1,1,bn=True,relu="learn")
        
        # Decoder
        self.conv7 = ConvTranspose2d(1024,1024,2,1,0,dropout=0.4,bn=True,relu="learn")
        self.conv8 = ConvTranspose2d(1024,512,4,2,1,bn=True,dropout=0.4,relu="learn")
        self.conv9 = ConvTranspose2d(512,256,4,2,1,bn=True, relu="learn")
        self.conv10 = ConvTranspose2d(256,128,4,2,1,bn=True, relu="learn")
        self.conv11 = ConvTranspose2d(128,64,4,2,1,bn=True, relu="learn")
        self.conv12 = ConvTranspose2d(64,64,4,2,1,bn=False, relu=False)

        #Output layer
        self.conv13 = Conv2d(64,5,3,padding=1)
                                 
                                        
 
    def forward(self, input):
#         x = F.relu(self.conv1(input))
# #         print('conv1', x.shape)
#         x = self.bn1(x)
        
#         x = F.relu(self.conv2(x))
# #         print('conv2', x.shape)
#         x = self.bn1(x)
        
#         x = F.relu(self.conv3(x))
# #         print('conv3', x.shape)
#         x = self.bn1(x)
        
#         x = F.relu(self.conv4(x))
# #         print('conv4', x.shape)
#         x = self.bn1(x)
        
        
#         x = F.relu(self.conv5(x))
# #         print('conv4', x.shape)
#         x = self.bn1(x)        
        
#         x = F.relu(self.conv6(x))
# #         print('conv5', x.shape)
#         output = self.bn1(x)        
#         sigmoid = nn.Sigmoid()    
#         output = sigmoid(output)         
        
        # print(input.shape)

        x = self.conv1(input)
        # print('conv1', x.shape)
        x = self.conv2(x)
        # print('conv2', x.shape)
        x = self.conv3(x)
        # print('conv3', x.shape)
        x = self.conv4(x)
        # print('conv4', x.shape)
        x = self.conv5(x)
        # print('conv5', x.shape)
        x = self.conv6(x)
        # print('conv6', x.shape)
        x = self.conv7(x)
        # print('conv7', x.shape)
        x = self.conv8(x)
        # print('conv8', x.shape)
        x = self.conv9(x)
        # print('conv9', x.shape)
        x = self.conv10(x)
        # print('conv10', x.shape)
        x = self.conv11(x)
        # print('conv11', x.shape)
        x = self.conv12(x)
        # print('conv12',x.shape)
        output = self.conv13(x)
        # print('conv13', output.shape)
        
        return output  

In [55]:
###   for smaller size of landsat ###########


class Discriminator(nn.Module):
    def __init__(self,landsat_inc):
        super(Discriminator, self).__init__()                               
        self.conv1 = Conv2d(landsat_inc*3,128,4,2,1,bn=False,relu=True,leak=0.2)
        self.conv2 = Conv2d(128,256,4,2,1,bn=True,relu=True,leak=0.2)
        self.conv3 = Conv2d(256,512,4,2,1,bn=True,relu=True,leak=0.2)
        self.conv4 = Conv2d(512,512,4,2,1,bn=True,relu=True,leak=0.2)
        self.conv5 = Conv2d(512,1,4,1,1,bn=False,relu=False)
        self.sigmoid = nn.Sigmoid()


    def forward(self, source,x):
#         print('dis input xnan', torch.isnan(x).sum())
#         print('dis input ynan', torch.isnan(y).sum())        
#         x = normalize(x)
#         y = normalize(y)        
        
        # x = F.relu(self.conv1(x))
        # y = F.relu(self.conv1(y))        
#         print('conv1', x.shape,y.shape)
        # x = self.bn1(x)
        # y = self.bn1(y)        
        
        # x = F.relu(self.conv2(x))
        # y = F.relu(self.conv2(y))         
#         print('conv2', x.shape,y.shape)
        # x = self.bn1(x)
        # y = self.bn1(y) 
        
        # x = F.relu(self.conv3(x))
        # y = F.relu(self.conv3(y))         
#         print('conv3', x.shape,y.shape)
        # x = self.bn1(x)
        # y = self.bn1(y) 
        
        # x_out = self.flt(x)
        # y_out = self.flt(y)       
#         print('flt', x_out.shape,y_out.shape)        

        
        # flt_out = torch.cat((x_out, y_out), 1)
#         print('fltout', flt_out.shape)         
#         x = F.relu(self.conv4(x))
#         print('conv4', x.shape)
#         x = self.bn1(x)
        
        
#         x = F.relu(self.conv5(x))
#         print('conv4', x.shape)
#         x = self.bn1(x)        
        
#         x = F.relu(self.conv6(x))
#         print('conv5', x.shape)
        # fc_out = self.fc1(flt_out)
#         print('fc_out',fc_out.shape)
        
        # sigmoid = nn.Sigmoid()
        # output = sigmoid(fc_out)  
#         print('output',output.shape)
        # print('x',x.shape)
        # print('source',source.shape)
        x = torch.cat((x,source),dim=1)
        # print('x',x.shape)
        x = self.conv1(x)
        # print('x',x.shape)
        x = self.conv2(x)
        # print('x',x.shape)
        x = self.conv3(x)
        # print('x',x.shape)
        x = self.conv4(x)
        # print('x',x.shape)
        output = self.conv5(x)
        # print('output',output.shape)
        output = self.sigmoid(output)
        # print('output',output.shape)
        
        return output  

In [56]:
path_to_gen = 'saved_models/gen'
path_to_dis = 'saved_models/dis'

# path_to_output = '../../../saved_outputs/landsat8_corn_2020'

In [57]:
def evaluate_gan(gen, dis, data_loader, dataset):
    
    gen.eval()
    dis.eval()
    
    batch_count = 0
    batch_gen_cGAN_loss = 0
    batch_gen_L1_loss = 0
    batch_gen_eval_loss = 0
    batch_dis_fake_loss = 0
    batch_dis_real_loss = 0
    batch_dis_eval_loss = 0
    gen_loss_eval = 0
    dis_loss_eval = 0
    batch_fooling_rate = 0
    batch_precision = 0
    batch_recall = 0
    batch_psnr = 0
    batch_ssim = 0
    batch_sam = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target =  (data.float()).to(device),target.float().to(device)
            

            dis_result_fake = dis(data, target)
            dis_real_loss = cGAN_loss(dis_result_fake, Variable((torch.zeros_like(dis_result_fake)).to(device)))
            
            gen_result = gen(data)
            dis_result = dis(data, gen_result)
            dis_fake_loss = cGAN_loss(dis_result, Variable((torch.rand(dis_result.size())*(1-0.8) + 0.8).to(device)))
            fooling_rate, precision, recall = _compute_classification_metrics(dis_result,dis_result_fake)
            batch_fooling_rate += fooling_rate
            batch_precision += precision
            batch_recall += recall
            dis_eval_loss = (dis_real_loss + dis_fake_loss)#/2.0
            
            gen_result = gen(data)
            dis_result = dis(data, gen_result)
            gen_eval_cGAN_loss = cGAN_loss(dis_result, Variable(torch.ones_like(dis_result).to(device)))
            gen_eval_L1_loss = gen_lambda * F.smooth_l1_loss(gen_result, target)
            gen_eval_loss = gen_eval_cGAN_loss + gen_eval_L1_loss
            psnr,ssim,sam = _compute_iqa_metrics(gen_result, target, reduction='mean')
            batch_psnr += psnr
            batch_ssim += ssim
            batch_sam += sam
            batch_count += 1;
            batch_gen_cGAN_loss += gen_eval_cGAN_loss
            batch_gen_eval_loss += gen_eval_loss
            
            
            
            batch_dis_fake_loss += dis_fake_loss
            batch_dis_real_loss += dis_real_loss
            batch_dis_eval_loss += dis_eval_loss
            
    gen_loss_eval = batch_gen_eval_loss/batch_count
    dis_loss_eval = batch_dis_eval_loss/batch_count    
    batch_fooling_rate = batch_fooling_rate/batch_count
    batch_precision = batch_precision/batch_count
    batch_recall = batch_recall/batch_count
    batch_psnr = batch_psnr/batch_count
    batch_ssim = batch_ssim/batch_count
    batch_sam = batch_sam/batch_count
    print('{} set: gen_loss: {}, dis_loss: {}\ndis_fooling_rate: {}, dis_precision: {}, dis_recall: {}\ngen_psnr: {}, gen_ssim: {}, gen_sam: {}\n'.format(dataset, gen_loss_eval, dis_loss_eval, batch_fooling_rate, batch_precision, batch_recall, batch_psnr, batch_ssim, batch_sam))
    if dataset == 'Validation':
      return gen_loss_eval, dis_loss_eval
    if dataset == 'Test':
      return gen_loss_eval, dis_loss_eval


In [69]:
def train_gan(gen, dis, opt_gen, opt_dis, epcs):
    for epoch in range(1, epcs+1):
        print('epoch', epoch)
        gen.train()
        dis.train()
        for batch_idx,(data,target) in enumerate(train_dataloader):
            data, target  = (data.float()).to(device), target.float().to(device)
            dis.zero_grad()
            gen_result = gen(data)
            dis_result_fake = dis(data, gen_result)          
            dis_fake_loss = cGAN_loss(dis_result_fake, Variable((torch.zeros_like(dis_result_fake)).to(device)))
            dis_result = dis(data, target)
            dis_real_loss = cGAN_loss(dis_result, Variable((torch.rand(dis_result.size())*(1-0.8) + 0.8).to(device)))
            dis_train_loss = (dis_real_loss + dis_fake_loss)#/2.0
            print('dis_train_loss',dis_train_loss)           
            fooling_rate, precision, recall = _compute_classification_metrics(dis_result,dis_result_fake)
            dis_train_loss.backward()
            opt_dis.step()
            gen.zero_grad()
            gen_result = gen(data)
            dis_result = dis(data, gen_result)
            gen_train_cGAN_loss = cGAN_loss(dis_result, Variable(torch.ones_like(dis_result).to(device)))
            gen_train_L1_loss = gen_lambda * F.smooth_l1_loss(gen_result, target)
            gen_train_loss = gen_train_cGAN_loss + gen_train_L1_loss
            psnr,ssim,sam = _compute_iqa_metrics(gen_result, target, reduction='mean')
            print('gen_train_loss',gen_train_loss) 
            gen_train_loss.backward()
            opt_gen.step()
        gen_lr_scheduler.step()
        disc_lr_scheduler.step()

        if (epoch>0):
            torch.save(gen, os.path.join(path_to_gen, "gen_%d.pt"%(epoch)))
            torch.save(dis, os.path.join(path_to_dis, "dis_%d.pt"%(epoch)))
        print('Train Epoch: {} Training gen Loss: {} dis Loss: {}'.format(epoch,gen_train_loss.item(),dis_train_loss.item()))
        gen_val_loss, dis_val_loss = evaluate_gan(gen, dis,val_dataloader,'Validation')
    return  gen_train_loss, dis_train_loss, gen_val_loss, dis_val_loss

In [70]:
def init_weights(module):
    if type(module) == nn.Linear:
        torch.nn.init.xavier_uniform_(module.weight)
        # torch.nn.init.xavier_normal_(module.weight, gain = 1.0)        
        module.bias.data.fill_(0.001)
    if type(module) == nn.Conv2d:    
        torch.nn.init.xavier_uniform(module.weight)
        module.bias.data.fill_(0.001)

In [71]:
landsat_inc = 5

# seq_length = len(timesteps)
hidden_dim = 20
epochs = 10   # 75
n_layers = 1
output_dim = 1

dp = 0.25


In [72]:
# timesteps, hidden_dim,sl, out_dim

gen = Generator(landsat_inc).to(device)
dis = Discriminator(landsat_inc).to(device)

# gen.apply(init_weights)
# dis.apply(init_weights)

cGAN_loss = nn.BCELoss().to(device)
gen_lambda = 0.1

opt_gen = torch.optim.Adam(gen.parameters(), lr=0.0003, betas=[0.5,0.999])
opt_dis = torch.optim.Adam(dis.parameters(), lr=0.0003, betas=[0.5,0.999], weight_decay=0.000001)
gen_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt_gen, gamma=0.99)
disc_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt_dis, gamma=0.99)

# optimizer = optim.SGD(model_vims.parameters(), lr=0.0001, weight_decay = 0.00001, momentum = 0.3)

Conv2d(
  (conv): Conv2d(10, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)


In [73]:
train_gen_loss, train_dis_loss, val_gen_loss, val_dis_loss = train_gan(gen, dis, opt_gen, opt_dis, epochs) # cl_l2,cl_l1, cl_l2,

epoch 1
batch_idx 0
dis_train_loss tensor(1.5763, grad_fn=<AddBackward0>)
gen_train_loss tensor(3.0178, grad_fn=<AddBackward0>)
batch_idx 1
dis_train_loss tensor(3.8015, grad_fn=<AddBackward0>)
gen_train_loss tensor(1.7703, grad_fn=<AddBackward0>)
batch_idx 2
dis_train_loss tensor(3.5406, grad_fn=<AddBackward0>)
gen_train_loss tensor(2.2063, grad_fn=<AddBackward0>)
batch_idx 3
dis_train_loss tensor(2.9830, grad_fn=<AddBackward0>)
gen_train_loss tensor(1.4648, grad_fn=<AddBackward0>)
batch_idx 4
dis_train_loss tensor(1.9150, grad_fn=<AddBackward0>)
gen_train_loss tensor(1.0576, grad_fn=<AddBackward0>)
batch_idx 5
dis_train_loss tensor(2.0524, grad_fn=<AddBackward0>)
gen_train_loss tensor(1.5870, grad_fn=<AddBackward0>)
batch_idx 6
dis_train_loss tensor(2.3278, grad_fn=<AddBackward0>)
gen_train_loss tensor(1.3015, grad_fn=<AddBackward0>)
batch_idx 7
dis_train_loss tensor(1.7712, grad_fn=<AddBackward0>)
gen_train_loss tensor(1.2510, grad_fn=<AddBackward0>)
batch_idx 8
dis_train_loss tenso